In [ ]:
import dlib
import cv2
import imutils
import base64
import boto3
import json
IOT_TOPIC = 'face/image'
iot = boto3.client('iot-data')

In [ ]:
# 開啟影片檔案
cap = cv2.VideoCapture(0)

# Dlib 的人臉偵測器
detector = dlib.get_frontal_face_detector()

# 以迴圈從影片檔案讀取影格，並顯示出來
while(cap.isOpened()):
  ret, frame = cap.read()

  # 偵測人臉
  face_rects, scores, idx = detector.run(frame, 0)

  # 取出所有偵測的結果
  count = 0
  for i, d in enumerate(face_rects):
    left = d.left()
    top = d.top()
    right = d.right()
    bottom = d.bottom()
    text = "%2.2f(%d)" % (scores[i], idx[i])
    
    # 以方框標示偵測的人臉
    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 4, cv2.LINE_AA)
    # 標示分數
    cv2.putText(frame, text, (left, top), cv2.FONT_HERSHEY_DUPLEX,
            0.7, (255, 255, 255), 1, cv2.LINE_AA)
    
    # 求長寬
    height = bottom - top
    width = right - left
    # 截圖長寬大於40
    if height > 40 and width > 40:
        print('face detected')
        face = frame[(top):(top + height),(left):(left + width)]
#         if height > width:
#             face = imutils.resize(face, width=128)
#         else:
#             face = imutils.resize(face, height=128)
        
        _, jpeg = cv2.imencode('.jpg', face)
        faceEncode = base64.b64encode(jpeg.tobytes())
        # Publish to IoT Core Topic
        print('Publish to AWS IoT Topic: face/image')
        response = iot.publish(
            topic= IOT_TOPIC,
            payload=json.dumps({
            'face': faceEncode.decode('utf-8')
        })
        )
    # 截圖太小
    else:
        print('face too small')
    
    


  # 顯示結果
  cv2.imshow("Face Detection", frame)
  
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()